# Mike Babb
# babbm@uw.edu
# Find anagrams
## Part 2: Generate and store the anagrams v1.0

In [ ]:
# standard libraries - installed by default
import collections
import datetime
import pickle
import sqlite3
import string
import os
import timeit

In [ ]:
# external libraries - not installed by default
import numpy as np
import pandas as pd

In [ ]:
from part_00_process_functions import load_pickle, build_db_conn

### set input and output paths

In [ ]:
# base file path
base_file_path = '/project/finding_anagrams'

In [ ]:
# input path
in_file_path = 'data'
in_file_path = os.path.join(base_file_path, in_file_path)

In [ ]:
# output db path and name
db_path = 'db'
db_path = os.path.join(base_file_path, db_path)

In [ ]:
if os.path.exists(db_path):
    pass
else:
    os.makedirs(db_path)

In [ ]:
db_name = 'words.db'

In [ ]:
# set up a sqlite3 database connection and cursor object
db_path_name = os.path.join(db_path, db_name)

### process control flags

In [ ]:
# Use numpy to perform matrix opertions and determine from/to and exact anagram relationships
# option 1 - work with the full char_matrix
# option 2 - create submatrices by word length
# option 3 - create submatrices by word length and letter
# option 4 - create submatrices by word length and least common two letters

matrix_extraction_option = 1

# max number of letters to slice to use for the generation of sub-matrices for
# option 4. More letters means more sub-matrices
n_common_letters = 3

# set write_data to true to store the generated list of anagrams
write_data = True

# set to None to include all letters
# test with a subset of letters by setting the letter_subset_list to ['q', 'x'] or 
# a different set of letters
#letter_subset_list = ['q', 'x', 's']
letter_subset_list = None

In [ ]:
# start a timer to record the entire operation
total_time_start = datetime.datetime.now()

### load input data

In [ ]:
# load the word_df, the words from Part 1
input_file_name = 'word_df.csv'
# build the file path
ipn = os.path.join(in_file_path, input_file_name)

# specify the datatypes of the columns using a dictionary
# because NA and NULL are reserved python words, but also words in our list of words,
# we need to specify the data types of the columns
dtype_dict = {'word': str,
              'lcase': str,
              'n_chars': int,
              'first_letter': str,
              'word_id': int,
              'word_group_id': int,
              'letter_group': str,
              'letter_group_ranked': str}

# read in the file and be careful of the NA and NULL values
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
word_df = pd.read_csv(filepath_or_buffer = ipn, sep = '\t',header = 0,
                          dtype=dtype_dict, na_values = '!!', keep_default_na=False)   

In [ ]:
word_df.tail()

In [ ]:
# extract the column of word ids as a numpy array
word_id_list = word_df['word_id'].to_numpy()    

In [ ]:
# create a dataframe with the letters sorted by the frequency of words that
# start with a particular letter
agg_word_df = word_df['first_letter'].groupby(word_df['first_letter']).agg(np.size).to_frame()

# set column names
agg_word_df.columns = ['word_count']

# reset the index to rename columns
agg_word_df = agg_word_df.reset_index()

# sort the dataframe by frequency
agg_word_df = agg_word_df.sort_values(by='word_count')

In [ ]:
agg_word_df.head(n=50)

In [ ]:
# extract the letters sorted by word frequency
sorted_first_letters = agg_word_df['first_letter'].tolist()

In [ ]:
# load the letter dictionary from part 1
in_file_name = 'letter_dict.pkl'
letter_dict = load_pickle(in_file_path = in_file_path, in_file_name=in_file_name)

In [ ]:
# load the word dictionary from part 1
in_file_name = 'word_dict.pkl'
word_dict = load_pickle(in_file_path = in_file_path, in_file_name=in_file_name)

In [ ]:
# load the char matrix from part 1
in_file_name = 'char_matrix.npy'
ipn = os.path.join(in_file_path, in_file_name)
char_matrix = np.load(file = ipn)

### extract sub-matrices

In [ ]:
word_df.head()

In [ ]:
# Option 1 does not take advantage of submatrices. Options 2, 3, and 4 do.

# the dictionary holding the sub-matrices
n_char_matrix_dict = {}

# by word length
word_length_list = sorted(word_df['n_chars'].unique().tolist())

# python dictionaries work by storing the hash values of objects
# Anything that can be hashed can be a dictionary key. 
# Computing the hash value of an object ahead of time can reduce dictionary access time.
# we'll compute the associated hash value of the tuple used to identify the sub-matrices.

word_id_n_char_matrix_dict = {}

In [ ]:
####
# BUILD OUT SUBMATRICES FOR OPTION 2
####
# Create submatrices based on the words with at least the same length as the focal word
if matrix_extraction_option == 2:
    loop_count = 0
    s_time = datetime.datetime.now()
    n_sub_matrices = len(word_length_list)
    print('...creating', n_sub_matrices, 'sub matrices')
        
    for i_nchars, n_chars in enumerate(word_length_list):
        # word id by character length
        curr_n_char_word_id_list  = word_df.loc[word_df['n_chars']>=n_chars, 'word_id'].to_numpy()
        
        #curr_n_char_word_id_list = curr_df['word_id']
        curr_char_matrix = char_matrix[curr_n_char_word_id_list, ]
                
        # use an empty string to form a consistent dictionary key 
        # across matrix_extraction_options
        key_value = (n_chars, '')
        key_value_hash = hash(key_value)
        n_char_matrix_dict[key_value_hash] = (curr_n_char_word_id_list, curr_char_matrix)        
    
    # store the tuple in the word_df
    word_df['word_id_n_char_matrix_key'] = word_df['n_chars'].map(lambda x: (x, ''))

In [ ]:
####
# BUILD OUT SUBMATRICES FOR OPTIONS 3 OR 4. 
####
if matrix_extraction_option in (3,4):
    loop_count = 0
    s_time = datetime.datetime.now()
    if matrix_extraction_option == 3:
        # by word length and first letter
        word_df['letter_selector'] = word_df['first_letter']
    else:   
        # by word length and n least common letters
        word_df['letter_selector'] = word_df['letter_group_ranked'].str[:n_common_letters]
    
    # store the tuple in the word_df
    # we have to use tuples because tuples are immutable - once created, they cannot be changed
    #https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences    
    word_df['word_id_n_char_matrix_key'] = tuple(zip(word_df['n_chars'], word_df['letter_selector']))
    
    # This is a combinatorial problem.
    # Limit the number of selections we need to make    
    letter_selector_df = word_df[['n_chars', 'letter_selector']].drop_duplicates()
    n_sub_matrices = len(letter_selector_df)
    print('...creating', n_sub_matrices, 'sub matrices')
    # this means n_sub_matrices are queried.
    # we can expedite this by only selecting certain word ids once, converting to a set,
    # and then storing that set based on the selection criteria.
    # many words are going to have the same least common characters, let's identify the
    # corresponding rows accordingly
    
    letter_selector_list = letter_selector_df['letter_selector'].unique().tolist()
    n_char_word_id_list_dict = {}
    ls_word_id_list_dict = {}
    
    for n_chars, letter_selector in zip(letter_selector_df['n_chars'], letter_selector_df['letter_selector']):
        
        # word id set by character length
        if n_chars in n_char_word_id_list_dict:
            # get the set if it already exists
            curr_n_char_word_id_set = n_char_word_id_list_dict[n_chars]
        else:
            # create the set if it does not exist
            curr_n_char_word_id_set = word_df.loc[(word_df['n_chars']>=n_chars) , 'word_id'].tolist()
            curr_n_char_word_id_set = set(curr_n_char_word_id_set)
            n_char_word_id_list_dict[n_chars] = curr_n_char_word_id_set

        # word id by letter selector
        if letter_selector in ls_word_id_list_dict:
            # get the set if it already exists
            curr_letter_select_word_id_set = ls_word_id_list_dict[letter_selector]
        else:
            # the set needs to be computed
            # build the oolumn selector using list comprehension
            column_selector = [letter_dict[curr_letter] for curr_letter in letter_selector]
            
            # create a true-false matrix where only certain columns, corresponding to
            # letter indices, have a value of 1 or more
            outcome = char_matrix[:, column_selector] > 0    
        
            # which rows in the above matrix evaluate to all True
            outcome_indices = np.all(a = outcome, axis = 1)
        
            # these indices match with the word_id_list, extract the subset        
            curr_letter_select_word_id_set = word_id_list[outcome_indices]
            curr_letter_select_word_id_set = set(curr_letter_select_word_id_set)
            ls_word_id_list_dict[letter_selector] = curr_letter_select_word_id_set
            
        # the set intersection of the curr_n_char_word_id_set and the
        # curr_letter_select_word_id_set are indices that feature a word of at
        # least a certain length and the characters of interest
        outcome_word_id_set = curr_n_char_word_id_set.intersection(curr_letter_select_word_id_set)
        # convert the set to an array
        outcome_word_id_list = np.array(list(outcome_word_id_set))
        
        # subset the char_matrix to get the sub matrix
        curr_char_matrix = char_matrix[outcome_word_id_list, ]

        # now, store that in the sub matrix dictionary
        key_value = (n_chars, letter_selector)
        key_value_hash = hash(key_value)
        n_char_matrix_dict[key_value_hash] = (outcome_word_id_list, curr_char_matrix)
        
        # simple progress display
        loop_count += 1
        if loop_count % 100 == 0:
            print(loop_count)

In [ ]:
if matrix_extraction_option != 1:    
    # populate the key_hash_field and then the word_id_n_char_matrix_dict
    # the word_id and the hashed (word_length, letters of interest) tuple are stored
    # in a dictionary to expedite comparison. This is a quick way to go from one id to another.    
    word_df['word_id_n_char_matrix_key_hash'] = word_df['word_id_n_char_matrix_key'].map(hash)
    for curr_word_id, curr_key_hash in zip(word_df['word_id'], word_df['word_id_n_char_matrix_key_hash']):
        word_id_n_char_matrix_dict[curr_word_id] = curr_key_hash
            
    e_time = datetime.datetime.now()
    p_time = e_time - s_time
    # how long did this pre-processing take?
    p_time = round(p_time.total_seconds(), 2)
    print('...sub-matrix extraction took', p_time, 'seconds...')

In [ ]:
len(word_id_n_char_matrix_dict)

In [ ]:
# let's examine what we've created, for processing options 3 and 4

In [ ]:
temp_focal_word = 'orange'

In [ ]:
if matrix_extraction_option in (3,4):
    temp_focal_word_id = word_df.loc[word_df['lcase']==temp_focal_word, 'word_id'].iloc[0]
    # the ID of the focal word
    print(temp_focal_word_id)
    # the hash corresponding to the tuple of the candidate word ids and the sub-matrix
    temp_focal_word_hash_id = word_id_n_char_matrix_dict[temp_focal_word_id]
    print(temp_focal_word_hash_id)
    # the candidate word ids and the sub matrix
    temp_word_id_list, temp_sub_matrix = n_char_matrix_dict[temp_focal_word_hash_id]
    print(temp_word_id_list.shape)
    print(temp_sub_matrix.shape)

In [ ]:
word_df.head()


### define a function to query the matrix, examine the output

In [ ]:
def get_values(word_id, word_dict, word_id_list, matrix_extraction_option,                
               n_char_matrix_dict, word_id_n_char_matrix_dict, char_matrix):
    """ FIND ANAGRAMS FOR A SPECIFIC USING word_id AND MATRIX COMPARISONS    
    """ 
    
    # A USEFUL WAY TO PROTOTYPE, TIME, AND DETERMINE THE
    # CORRECTNESS OF PROGRAM OPERATION AND OUTPUT

    # get information data based on word id
    cw, cw_length, cfl, clg, clgr = word_dict[word_id]   
    
    if matrix_extraction_option == 1:
        outcome = char_matrix - char_matrix[word_id, ]
        
    if matrix_extraction_option in (2, 3, 4):        
        key_hash = word_id_n_char_matrix_dict[word_id]
        
        cw_id_list, curr_char_matrix = n_char_matrix_dict[key_hash]
        # subtract the curr_test_vector from every row in the matrix
        # this produces a new matrix.        
        new_word_id = cw_id_list==word_id        
        outcome = curr_char_matrix - curr_char_matrix[new_word_id, ]
        
    # compute the score by finding where rows, across all columns, are GTE 0
    outcome_indices = np.all(outcome >= 0, axis = 1)
    outcome = None        

    # extract anagrams based on index values
    if matrix_extraction_option == 1:
        outcome_word_id_list = word_id_list[outcome_indices]
    else:
        outcome_word_id_list = cw_id_list[outcome_indices]    
    
    output_list = np.zeros(shape = (len(outcome_word_id_list), 2),  dtype=np.int32)
    
    # update the output list with the word_id_list - these are from/parent words    
    output_list[:, 0] = outcome_word_id_list
    
    # update with the word_id - this is the to/child word
    output_list[:, 1] = word_id
        
    return output_list

In [ ]:
# test with the word 'quiet', id 160875
curr_word_id = 160875
curr_word, curr_word_length, curr_first_letter, curr_letter_group, curr_letter_group_ranked = word_dict[curr_word_id] 

In [ ]:
curr_word

In [ ]:
#word_id_n_char_matrix_dict[160875]

In [ ]:
output = get_values(word_id = curr_word_id, word_dict = word_dict, word_id_list = word_id_list, 
                    matrix_extraction_option = matrix_extraction_option,
                    n_char_matrix_dict = n_char_matrix_dict,
                    word_id_n_char_matrix_dict = word_id_n_char_matrix_dict,
                    char_matrix = char_matrix)

In [ ]:
# how many parent/from words were found for the word 'quiet'?
len(output)

In [ ]:
# this is an array of from words to the word 'quiet'
output

In [ ]:
# and those words are...
word_list = word_df.loc[word_df['word_id'].isin(output[:, 0]), 'lcase'].tolist()

In [ ]:
word_list

In [ ]:
# we've tested with one word, let's time many evaluations to get a sense of how quickly 
# the current matrix_extraction_option executes
# use the timeit() function to evaluate how long, on average, a single matrix operation
# takes to complete
code_snippet = """get_values(word_id = curr_word_id, word_dict = word_dict, word_id_list = word_id_list, 
                    matrix_extraction_option = matrix_extraction_option,
                    n_char_matrix_dict = n_char_matrix_dict,
                    word_id_n_char_matrix_dict = word_id_n_char_matrix_dict,
                    char_matrix = char_matrix)"""

In [ ]:
n_trials = 1000
total_time = timeit.timeit(code_snippet,
              number=n_trials, globals=globals())

In [ ]:
# average number of seconds per trial
total_time / n_trials

### estimate total number of from/to word pairs

In [ ]:
# how many anagrams are there?
# let's estimate the number of anagrams by assuming that the number of
# parent/from words is a function of word length. 
# let's sample 10 words of each word length, compute the number of from/parent anagrams
# for each word in the sample, compute the min, mean, and max, and apply those values
# to the numbers of words by length and multiply accordingly
# this will give us very generous upper bound of anagram pairs

In [ ]:
# list of the number of characters per word
n_char_list = sorted(word_df['n_chars'].unique().tolist())

In [ ]:
# enumerate and sample
output_list = []
for i_n_char, n_char in enumerate(n_char_list):
    curr_id_list = word_df.loc[word_df['n_chars']==n_char, 'word_id'].to_numpy()
    # sample with replacement
    sample_id_list = np.random.choice(a = curr_id_list, size = 10, replace = True)
    for sid in sample_id_list:
        output = get_values(word_id = sid, word_dict = word_dict, word_id_list = word_id_list, 
                    matrix_extraction_option = matrix_extraction_option,
                    n_char_matrix_dict = n_char_matrix_dict,
                    word_id_n_char_matrix_dict = word_id_n_char_matrix_dict,
                    char_matrix = char_matrix)
        curr_from_words = len(output)
        curr_output = [n_char, curr_from_words]
        output_list.append(curr_output)    

In [ ]:
len(output_list)

In [ ]:
# make a dataframe
pos_df = pd.DataFrame(data = output_list, columns = ['n_chars', 'n_from_words'])

In [ ]:
# minimum, max, and mean number of from words
agg_pos_df = pos_df.groupby('n_chars').agg([np.min, np.max, np.mean])

In [ ]:
agg_pos_df.head()

In [ ]:
agg_pos_df.columns = ['min_n_from_words', 'max_n_from_words', 'mean_n_from_words']

In [ ]:
# let's aggregate by number of letters per word, and then join
n_word_length_df = word_df['n_chars'].groupby(word_df['n_chars']).agg(np.size).to_frame()
n_word_length_df.columns = ['n_words']

In [ ]:
n_pos_df = pd.merge(left = n_word_length_df, right = agg_pos_df, left_index = True,
                   right_index = True)

In [ ]:
n_pos_df['n_tot_max_anagrams'] = n_pos_df['n_words'] * n_pos_df['max_n_from_words']
n_pos_df['n_tot_mean_anagrams'] = n_pos_df['n_words'] * n_pos_df['mean_n_from_words']

In [ ]:
# set the upper bound of anagrams as the midway point
# between the mean and the max of the estimated number of anagrams
n_possible_anagrams = (n_pos_df['n_tot_mean_anagrams'].sum() + n_pos_df['n_tot_max_anagrams'].sum()) / 2

In [ ]:
# round and convert to integer
n_possible_anagrams = int(np.round(n_possible_anagrams, 0))

In [ ]:
# this number will be used to create an array that will hold the from/to pairs
n_possible_anagrams

### discover from/to word pairs

In [ ]:
# initialize counters to count the number of to (child words) from a focal word.
# we could do this in post-processing, but the data are already in memory and it's a simple 
# calculation to make.
# we want to minimize the number of trips through our data.

# the number of candidate words examined for each focal word

# a list to hold the dataframes generated for each letter
proc_time_df_list = []

# subset the list of leters
if letter_subset_list:
    letters = letter_subset_list[:]
else:
    letters = sorted_first_letters

anagram_pair_count = 0 
# use numpy to pre-allocate an array that will be updated while enumerating. 
# this eliminates list.append() calls
# note: I am guessing that there are 150M anagram pairs. 

output_list = np.full(shape = (n_possible_anagrams, 3), fill_value = -1,  dtype=np.int32)

for i_cl, curr_letter in enumerate(letters):
    # enumerate by each letter
    # this isn't absolutely necessary, we could just enumerate by word id, 
    # but for testing and development, letters are a handy way to chunk up the data. 

    # this dictionary will store the calculations for each letter
    proc_time_dict = {}    
    
    # the list of words that start with the focal letter     
    curr_word_df = word_df.loc[word_df['first_letter'] == curr_letter, :]
    
    # sort the dataframe by n_chars and letter_selector, if it exists.
    # this will cut down on dictionary lookups for matrix_extraction_types 3 and 4.    
    if 'letter_selector' in word_df.columns.tolist():
        curr_word_df = curr_word_df.sort_values(by = ['n_chars', 'letter_selector'])
    else:
        curr_word_df = curr_word_df.sort_values(by = ['n_chars'])
        
    curr_word_id_list = curr_word_df['word_id'].tolist()
    
    n_curr_words = '{:,}'.format(len(curr_word_df))    
    print('...finding parent anagrams for', n_curr_words, 'words that start with', curr_letter)               
    
    # enumerate by word id, working with integers is faster than words    
    for i_wi, word_id in enumerate(curr_word_id_list):            
        # start timing to record processing for each word            
        s_time = datetime.datetime.now()
        
        # get the current word length, from the word id
        #to_word, to_word_length, curr_first_letter, clg, clgr = word_dict[word_id]   
        to_word_length = word_dict[word_id][1]
        
        if matrix_extraction_option == 1:
            outcome = char_matrix - char_matrix[word_id, ]
            n_possible_words = char_matrix.shape[0]            

        if matrix_extraction_option in (2, 3, 4):        
            
            # get the tuple associated with the word id
            # much faster to look up stored values for the hash value than it is to 
            # only look up if the hash value has changed            
            key_hash = word_id_n_char_matrix_dict[word_id]                
            # get the possible candidate word_ids and char matrix
            curr_word_id_index_list, curr_char_matrix = n_char_matrix_dict[key_hash]                                
        
            # how many candidates?
            n_possible_words = len(curr_word_id_index_list)
        
            # subtract the curr_test_vector from every row in the matrix
            # this produces a new matrix.        
            new_word_id = curr_word_id_index_list == word_id            
            outcome = curr_char_matrix - curr_char_matrix[new_word_id, ]
                        
        # compute the score by finding where rows, across all columns, are GTE 0
        outcome_indices = np.all(outcome >= 0, axis = 1)
        outcome = None        
        
        # extract anagrams based on same index values
        if matrix_extraction_option == 1:
            outcome_word_id_list = word_id_list[outcome_indices].tolist()
        else:                
            outcome_word_id_list = curr_word_id_index_list[outcome_indices].tolist()
            
        outcome_indices = None               
        
        # if the outcome is greater than or equal to zero, then the current word is an
        # anagram of the other word    
        # a value  >= 0 means that the current word contains the exact same number of focal letters
        # mite --> time or miter --> time
        # a value >= 1 means that current word contains at least the same number of focal letters
        # terminator --> time
        # a value of <=-1 means that the current word does not have the 
        # correct number of letters and is therefore not an anagram.
        # trait <> time        

        # number of parent words found
        n_from_words = len(outcome_word_id_list)

        if n_from_words > 1:
            
            # we have matches
            # the focal word   
                                    
            # enumerate the from/parent words
            # from word length
            from_word_length_list = [word_dict[from_word_id][1] for from_word_id in outcome_word_id_list]
            same_word_length_list = [1 if fwl == to_word_length else 0 for fwl in from_word_length_list]
            
            new_anagram_pair_count = anagram_pair_count + len(from_word_length_list)
            # the from words
            output_list[anagram_pair_count:new_anagram_pair_count, 0] = outcome_word_id_list        
            # the to word
            output_list[anagram_pair_count:new_anagram_pair_count, 1] = word_id                                            
            # same length
            output_list[anagram_pair_count:new_anagram_pair_count, 2] = same_word_length_list                                                                       
            # set the anagram pair count
            anagram_pair_count = new_anagram_pair_count
                    
                
        del outcome_word_id_list
            
        # record the time for the word
        e_time = datetime.datetime.now()
        p_time = e_time - s_time    
        p_time = p_time.total_seconds()

        proc_time_dict[word_id] = (p_time, n_from_words, n_possible_words)       
    
    # create a dataframe from the proc_time_dict
    proc_time_df = pd.DataFrame.from_dict(data=proc_time_dict, orient='index')
    proc_time_df = proc_time_df.reset_index()
    proc_time_df.columns = ['word_id', 'n_seconds', 'n_from_words', 'n_candidates']                
    
    # display processing time for the current letter
    total_proc_time = round(proc_time_df['n_seconds'].sum(), 2)
    print('...finding parent anagrams for', curr_letter, 'words took', total_proc_time, 'seconds...')
    
    proc_time_df_list.append(proc_time_df)

### shape and store output data

In [ ]:
# truncate the output array to only include indices with a from/to word pair
output_indices = np.all(output_list >= 0, axis = 1)
output_list = output_list[output_indices, ]
del output_indices

In [ ]:
# how many anagram pairs were found?
n_total_anagrams = len(output_list)
n_total_anagrams_formatted = '{:,}'.format(n_total_anagrams)
print('...total anagrams', n_total_anagrams_formatted)

In [ ]:
## count the number of to words, same length to words, and different length to words for each word using counters
# https://docs.python.org/3/library/collections.html#collections.Counter
# number of to words

In [ ]:
# the count of to words
to_word_counter = collections.Counter(output_list[:, 0])

In [ ]:
# the count of same length to words
curr_indices = output_list[:, 2] == 1
curr_output_list = output_list[curr_indices, 0]

In [ ]:
slt_word_counter = collections.Counter(curr_output_list)

In [ ]:
# the count of different length to words
# substract the counters from each other
dlt_word_counter = to_word_counter - slt_word_counter

In [ ]:
# create database connection objects
db_conn = build_db_conn(db_path = db_path, db_name = db_name)
db_cursor = db_conn.cursor()

### write anagram pairs to SQLite

In [ ]:
len(output_list)

In [ ]:
# write the anagram pairs to the database
if write_data:

    # let's write to the SQLite database in chunks of 1M records
    break_point_list = list(range(0, len(output_list), 1000000))
    # add the last bit of records
    if break_point_list[-1] < len(output_list):
        break_point_list.append(len(output_list))
    
    # drop the anagrams table if it previously exists
    sql = 'drop table if exists anagrams;'
    
    print('...dropping previous table...')
    # send the sql statement to the database and commit the changes
    db_cursor.execute(sql)
    db_conn.commit()

    # create the anagrams table
    sql = 'create table anagrams ( from_word_id integer, to_word_id integer, same_word_length integer);'

    # execute the statement and commit changes    
    db_cursor.execute(sql)
    db_conn.commit()
        
    # objects to record write time
    db_write_time_list = []
    db_write_time_start = datetime.datetime.now()
    
    # create a sql statement that we'll use to insert values.
    print('...beginning to add anagram word pairs...')
    base_sql = 'insert into anagrams values (?,?,?)'    
    
    insert_count = 0    
    curr_db_write_time_start = datetime.datetime.now()
    for i_bp, bp in enumerate(break_point_list[:-1]):
        # slice the output list of word id pairs, convert to a python list
        # the numpy.int data type is not compatable with sqlite.
        # the cursor.executemany() is a quick way to write a lot of data.
        next_bp = break_point_list[i_bp + 1]
        
        # converting the entire output_list to a python list adds too much overheard.
        test_output_list = output_list[bp:next_bp, ].tolist()
        
        # use the executemany() function to write records
        #https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.executemany
        db_cursor.executemany(base_sql, test_output_list)
        
        # commit changes every 10M records        
        if next_bp % 10000000 == 0:
            print('...commiting changes:', '{:,}'.format(next_bp), 'records')
            db_conn.commit()                       
            curr_db_write_time_end = datetime.datetime.now()
            curr_db_write_time_proc = curr_db_write_time_end - curr_db_write_time_start            
            curr_db_write_time_proc = curr_db_write_time_proc.total_seconds()
            curr_db_write_time_start = datetime.datetime.now()
            db_write_time_list.append(curr_db_write_time_proc)                                      

            # compute average write time, display after 1M writes
            mean_write_time = np.mean(db_write_time_list)

            # compute ETA            
            n_seconds = (n_total_anagrams / 10000000) * mean_write_time
            add_seconds = datetime.timedelta(seconds = n_seconds)
            eta_write_complete = db_write_time_start + add_seconds            
            eta_write_complete = eta_write_complete.strftime(format = "%m/%d/%Y, %H:%M:%S")

            mean_write_time = round(mean_write_time, 3)
            print('...average write time per 10M records:', mean_write_time, 'seconds...')
            print('...estimated write complete time:', eta_write_complete)
        
    # commit the last round of changes
    print('...commiting changes:', '{:,}'.format(len(test_output_list)), 'records')
    db_conn.commit()
    
    # compute total write times
    db_write_time_end = datetime.datetime.now()
    db_write_time_proc = db_write_time_end - db_write_time_start
    db_write_time_proc = db_write_time_proc.total_seconds() / 60
    db_write_time_proc = round(db_write_time_proc, 2)
    print('...writing to db took', db_write_time_proc, 'minutes')
    
    del test_output_list
    
# remove the list of from/to word pairs
del output_list    

### store number of from/to word pairs and time related to processing

In [ ]:
# create a dataframe with the processing times
proc_time_df = pd.concat(proc_time_df_list)

# drop columns related to data processing
drop_col_names = ['letter_selector', 'word_id_n_char_matrix_key',
                  'word_id_n_char_matrix_key_hash']
curr_col_names = word_df.columns.tolist()
for dcn in drop_col_names:    
    if dcn in curr_col_names:
        word_df = word_df.drop(dcn, axis = 1)

# merge the word_df and the proc_time_df dataframes to get the processing time per word
word_df = pd.merge(left=word_df, right = proc_time_df)

# now, use the map function to get the number of from/to words and the number of
# candidate words for each word
word_df['n_to_words'] = word_df['word_id'].map(to_word_counter)
word_df['n_slt_words'] = word_df['word_id'].map(slt_word_counter)
word_df['n_dlt_words'] = word_df['word_id'].map(dlt_word_counter)
#word_df['n_candidates'] = word_df['word_id'].map(n_possible_words_counter)

# record the matrix extraction option
word_df['matrix_extraction_option'] = matrix_extraction_option

In [ ]:
# rearrange columns
col_names = ['word','lcase','n_chars','first_letter','word_id',
             'word_group_id','letter_group','letter_group_ranked','n_seconds',
             'n_from_words','n_to_words','n_slt_words','n_dlt_words','n_candidates',
             'matrix_extraction_option']
word_df = word_df[col_names]

In [ ]:
# output table name
table_name = 'words_me_' + str(matrix_extraction_option).zfill(2)
# write the processing option table
word_df.to_sql(name=table_name, con=db_conn, if_exists='replace', index = False)    
# write the words table
word_df.to_sql(name='words', con=db_conn, if_exists='replace', index = False)    
    
# close the connection
db_cursor.close()
db_conn.close()

In [ ]:
anagram_discovery_time = word_df['n_seconds'].sum()
anagram_discovery_time = anagram_discovery_time / 60
anagram_discovery_time = round(anagram_discovery_time, 2)

In [ ]:
print('...anagram discovery time:', anagram_discovery_time, 'minutes')

In [ ]:
# record the total time
total_time_end = datetime.datetime.now()
total_time_proc = total_time_end - total_time_start
total_time_proc = total_time_proc.total_seconds()
total_time_proc = total_time_proc / 60
total_time_proc = round(total_time_proc, 2)

In [ ]:
print('...total processing time:', total_time_proc, 'minutes')